# Satellite Imagery Similarity Search

This notebook demonstrates how to use the H3 Satellite Search system to find similar locations based on satellite imagery. We'll explore different search methods:

1. Search by coordinates
2. Search by example tile
3. Search by multiple examples (e.g., grain elevators at different ports)
4. Visualize search results

## Setup

First, let's import the necessary libraries and set up the environment.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import h3
import geopandas as gpd
from shapely.geometry import Point, Polygon
from PIL import Image
import rasterio
from rasterio.plot import show
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue

# Add the parent directory to the path
sys.path.append('..')

# Import project modules
from utils.ports import get_port_coordinates, get_port_bbox, get_all_ports
from utils.tiling import point_to_h3, h3_to_polygon
from utils.indexing import connect_to_qdrant

## Connect to Vector Database

Connect to the Qdrant vector database where our embeddings are stored.

In [ ]:
# Connect to Qdrant
vector_db_path = "../data/processed/vector_db"
client = connect_to_qdrant(vector_db_path)
print(f"Connected to vector database at {vector_db_path}")

# Check collection info
collection_name = "satellite_embeddings"
collection_info = client.get_collection(collection_name)
print(f"Collection name: {collection_info.name}")
print(f"Vector size: {collection_info.config.params.vectors.size}")
print(f"Distance function: {collection_info.config.params.vectors.distance}")
print(f"Number of vectors: {collection_info.points_count}")

## Load Tile Data

Load the tile data for all ports.

In [ ]:
# Load tile data
tiles_path = "../data/processed/all_tiles.geojson"
all_tiles = gpd.read_file(tiles_path)
print(f"Loaded {len(all_tiles)} tiles from {tiles_path}")
all_tiles.head()

## Visualize All Tiles

Let's visualize all the tiles for all ports on a map.

In [ ]:
# Create a map centered on China
m = folium.Map(location=[35, 120], zoom_start=5)

# Add tiles for each port with different colors
colors = {
    "tianjin": "red",
    "dalian": "blue",
    "shanghai": "green",
    "qingdao": "purple"
}

# Add tiles to the map
for port in get_all_ports():
    port_tiles = all_tiles[all_tiles["port"] == port]
    
    # Add port marker
    port_coords = get_port_coordinates(port)
    folium.Marker(
        location=[port_coords["lat"], port_coords["lng"]],
        popup=port.capitalize(),
        icon=folium.Icon(color=colors[port], icon="info-sign")
    ).add_to(m)
    
    # Add tiles
    for _, row in port_tiles.iterrows():
        folium.GeoJson(
            row["geometry"],
            style_function=lambda x, color=colors[port]: {
                "fillColor": color,
                "color": color,
                "weight": 1,
                "fillOpacity": 0.3
            },
            tooltip=f"Port: {port}, H3: {row['h3_index']}"
        ).add_to(m)

m

## Method 1: Search by Coordinates

Let's search for similar locations based on coordinates. We'll find the H3 tile containing the coordinates, get its embedding, and search for similar tiles.

In [ ]:
def search_by_coordinates(lat, lng, limit=5, port_filter=None):
    """Search for similar locations based on coordinates."""
    # Find the H3 tile containing the coordinates
    h3_index = h3.latlng_to_cell(lat, lng, 6)
    print(f"Coordinates ({lat}, {lng}) are in H3 tile: {h3_index}")
    
    # Check if the tile exists in our dataset
    matching_tiles = all_tiles[all_tiles["h3_index"] == h3_index]
    
    if len(matching_tiles) == 0:
        print(f"No tile found for H3 index {h3_index}")
        return None
    
    # Get the port for the tile
    port = matching_tiles.iloc[0]["port"]
    print(f"Tile belongs to port: {port}")
    
    # Try to load the embedding from file
    embedding_path = f"../data/embeddings/{port}/{h3_index}.npy"
    if os.path.exists(embedding_path):
        embedding = np.load(embedding_path)
        print(f"Loaded embedding from file: {embedding_path}")
    else:
        # Search for the embedding in the database
        results = client.search(
            collection_name=collection_name,
            query_filter=Filter(
                must=[
                    FieldCondition(
                        key="h3_index",
                        match=MatchValue(value=h3_index)
                    )
                ]
            ),
            query_vector=[0.0] * 128,  # Dummy vector, we just want to retrieve the stored vector
            limit=1
        )
        
        if not results:
            print(f"No embedding found for H3 index {h3_index}")
            return None
        
        embedding = np.array(results[0].vector)
        print(f"Retrieved embedding from database")
    
    # Create filter if needed
    filter_obj = None
    if port_filter:
        filter_obj = Filter(
            must=[
                FieldCondition(key="port", match=MatchValue(value=port_filter))
            ]
        )
    
    # Search for similar tiles
    results = client.search(
        collection_name=collection_name,
        query_vector=embedding.tolist(),
        limit=limit,
        query_filter=filter_obj
    )
    
    return results

# Example: Search for locations similar to a point in Tianjin port
tianjin_coords = get_port_coordinates("tianjin")
results = search_by_coordinates(tianjin_coords["lat"], tianjin_coords["lng"], limit=5)

# Display results
if results:
    print("\nSearch results:")
    for i, result in enumerate(results):
        print(f"Result {i+1}:")
        print(f"  H3 Index: {result.payload['h3_index']}")
        print(f"  Port: {result.payload['port']}")
        print(f"  Similarity Score: {result.score:.4f}")

## Visualize Search Results

Let's visualize the search results on a map.

In [ ]:
def visualize_results(results, query_lat=None, query_lng=None):
    """Visualize search results on a map."""
    if not results:
        print("No results to visualize")
        return
    
    # Get the center of the first result
    first_h3 = results[0].payload["h3_index"]
    center = h3.cell_to_latlng(first_h3)
    center_lat, center_lng = center[0], center[1]
    
    # Create a map centered on the first result
    m = folium.Map(location=[center_lat, center_lng], zoom_start=8)
    
    # Add query point if provided
    if query_lat and query_lng:
        folium.Marker(
            location=[query_lat, query_lng],
            popup="Query Point",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)
    
    # Add results to the map
    for i, result in enumerate(results):
        # Get the H3 polygon
        h3_index = result.payload["h3_index"]
        h3_polygon = h3_to_polygon(h3_index)
        
        # Get the center of the H3 cell
        center = h3.cell_to_latlng(h3_index)
        lat, lng = center[0], center[1]
        
        # Add the polygon to the map
        folium.GeoJson(
            h3_polygon,
            style_function=lambda x, score=result.score: {
                "fillColor": "green",
                "color": "green",
                "weight": 2,
                "fillOpacity": min(0.8, max(0.2, score))
            },
            tooltip=f"Rank: {i+1}, Score: {result.score:.4f}, Port: {result.payload['port']}"
        ).add_to(m)
        
        # Add a marker for the center of the tile
        folium.Marker(
            location=[lat, lng],
            popup=f"Rank: {i+1}, Score: {result.score:.4f}",
            icon=folium.Icon(color="blue", icon=str(i+1))
        ).add_to(m)
    
    return m

# Visualize the results
if results:
    m = visualize_results(results, query_lat=tianjin_coords["lat"], query_lng=tianjin_coords["lng"])
    m

## Method 2: Search by Example Tile

Let's search for similar locations based on an example tile. We'll use a tile from one port and find similar tiles in other ports.

In [ ]:
def search_by_example_tile(h3_index, limit=5, exclude_port=None):
    """Search for similar locations based on an example tile."""
    # Check if the tile exists in our dataset
    matching_tiles = all_tiles[all_tiles["h3_index"] == h3_index]
    
    if len(matching_tiles) == 0:
        print(f"No tile found for H3 index {h3_index}")
        return None
    
    # Get the port for the tile
    port = matching_tiles.iloc[0]["port"]
    print(f"Example tile belongs to port: {port}")
    
    # Try to load the embedding from file
    embedding_path = f"../data/embeddings/{port}/{h3_index}.npy"
    if os.path.exists(embedding_path):
        embedding = np.load(embedding_path)
        print(f"Loaded embedding from file: {embedding_path}")
    else:
        # Search for the embedding in the database
        results = client.search(
            collection_name=collection_name,
            query_filter=Filter(
                must=[
                    FieldCondition(
                        key="h3_index",
                        match=MatchValue(value=h3_index)
                    )
                ]
            ),
            query_vector=[0.0] * 128,  # Dummy vector, we just want to retrieve the stored vector
            limit=1
        )
        
        if not results:
            print(f"No embedding found for H3 index {h3_index}")
            return None
        
        embedding = np.array(results[0].vector)
        print(f"Retrieved embedding from database")
    
    # Create filter if needed
    filter_obj = None
    if exclude_port:
        filter_obj = Filter(
            must_not=[
                FieldCondition(key="port", match=MatchValue(value=exclude_port))
            ]
        )
    
    # Search for similar tiles
    results = client.search(
        collection_name=collection_name,
        query_vector=embedding.tolist(),
        limit=limit,
        query_filter=filter_obj
    )
    
    return results

# Example: Search for locations similar to a specific tile in Shanghai port
# First, let's find a tile in Shanghai
shanghai_tiles = all_tiles[all_tiles["port"] == "shanghai"]
if len(shanghai_tiles) > 0:
    example_tile = shanghai_tiles.iloc[0]["h3_index"]
    print(f"Using example tile from Shanghai: {example_tile}")
    
    # Search for similar tiles in other ports
    results = search_by_example_tile(example_tile, limit=5, exclude_port="shanghai")
    
    # Display results
    if results:
        print("\nSearch results:")
        for i, result in enumerate(results):
            print(f"Result {i+1}:")
            print(f"  H3 Index: {result.payload['h3_index']}")
            print(f"  Port: {result.payload['port']}")
            print(f"  Similarity Score: {result.score:.4f}")

In [ ]:
# Visualize the results
if results:
    # Get the coordinates of the example tile
    center = h3.cell_to_latlng(example_tile)
    example_lat, example_lng = center[0], center[1]
    
    m = visualize_results(results, query_lat=example_lat, query_lng=example_lng)
    m

## Method 3: Search by Multiple Examples

Let's search for similar locations based on multiple examples. This is useful for finding specific types of infrastructure (e.g., grain elevators) across different ports.

In [ ]:
def search_by_multiple_examples(coordinates_list, limit=5):
    """Search for similar locations based on multiple example coordinates."""
    # Convert coordinates to H3 indices
    h3_indices = []
    for lat, lng in coordinates_list:
        h3_index = h3.latlng_to_cell(lat, lng, 6)
        h3_indices.append(h3_index)
        print(f"Coordinates ({lat}, {lng}) are in H3 tile: {h3_index}")
    
    # Get embeddings for each H3 index
    embeddings = []
    
    for h3_index in h3_indices:
        # Try to load the embedding from file
        for port in get_all_ports():
            embedding_path = f"../data/embeddings/{port}/{h3_index}.npy"
            if os.path.exists(embedding_path):
                embedding = np.load(embedding_path)
                embeddings.append(embedding)
                print(f"Found embedding for H3 index {h3_index} in port {port}")
                break
        else:
            # Search for the embedding in the database
            results = client.search(
                collection_name=collection_name,
                query_filter=Filter(
                    must=[
                        FieldCondition(
                            key="h3_index",
                            match=MatchValue(value=h3_index)
                        )
                    ]
                ),
                query_vector=[0.0] * 128,  # Dummy vector, we just want to retrieve the stored vector
                limit=1
            )
            
            if results:
                embeddings.append(np.array(results[0].vector))
                print(f"Retrieved embedding from database for H3 index {h3_index}")
            else:
                print(f"No embedding found for H3 index {h3_index}")
    
    if not embeddings:
        print("No embeddings found for the provided coordinates")
        return None
    
    # Average the embeddings
    avg_embedding = np.mean(embeddings, axis=0)
    
    # Search for similar tiles using the average embedding
    results = client.search(
        collection_name=collection_name,
        query_vector=avg_embedding.tolist(),
        limit=limit
    )
    
    return results

# Example: Search for locations similar to multiple examples
# Let's use coordinates of potential grain elevators in different ports
example_coordinates = [
    [39.0150, 117.7200],  # Tianjin
    [38.9600, 121.6400],  # Dalian
    [31.3500, 121.5000]   # Shanghai
]

results = search_by_multiple_examples(example_coordinates, limit=10)

# Display results
if results:
    print("\nSearch results:")
    for i, result in enumerate(results):
        print(f"Result {i+1}:")
        print(f"  H3 Index: {result.payload['h3_index']}")
        print(f"  Port: {result.payload['port']}")
        print(f"  Similarity Score: {result.score:.4f}")

In [ ]:
# Visualize the results on a larger map
if results:
    # Create a map centered on China
    m = folium.Map(location=[35, 120], zoom_start=5)
    
    # Add example points
    for lat, lng in example_coordinates:
        folium.Marker(
            location=[lat, lng],
            popup="Example Point",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)
    
    # Add results to the map
    for i, result in enumerate(results):
        # Get the H3 polygon
        h3_index = result.payload["h3_index"]
        h3_polygon = h3_to_polygon(h3_index)
        
        # Get the center of the H3 cell
        center = h3.cell_to_latlng(h3_index)
        lat, lng = center[0], center[1]
        
        # Add the polygon to the map
        folium.GeoJson(
            h3_polygon,
            style_function=lambda x, score=result.score: {
                "fillColor": "green",
                "color": "green",
                "weight": 2,
                "fillOpacity": min(0.8, max(0.2, score))
            },
            tooltip=f"Rank: {i+1}, Score: {result.score:.4f}, Port: {result.payload['port']}"
        ).add_to(m)
        
        # Add a marker for the center of the tile
        folium.Marker(
            location=[lat, lng],
            popup=f"Rank: {i+1}, Score: {result.score:.4f}, Port: {result.payload['port']}",
            icon=folium.Icon(color="blue", icon=str(i+1))
        ).add_to(m)
    
    m

## Visualize Satellite Imagery

Let's visualize the satellite imagery for some of the search results.

In [ ]:
def visualize_tile_imagery(h3_index):
    """Visualize the satellite imagery for a specific H3 tile."""
    # Check if the tile exists in our dataset
    matching_tiles = all_tiles[all_tiles["h3_index"] == h3_index]
    
    if len(matching_tiles) == 0:
        print(f"No tile found for H3 index {h3_index}")
        return None
    
    # Get the port for the tile
    port = matching_tiles.iloc[0]["port"]
    
    # Construct the path to the imagery file
    imagery_path = f"../data/raw/{port}/{h3_index}.tif"
    
    if not os.path.exists(imagery_path):
        print(f"No imagery found for tile {h3_index}")
        return None
    
    # Load and display the imagery
    with rasterio.open(imagery_path) as src:
        # Read the first three bands (RGB)
        rgb = src.read([1, 2, 3])
        
        # Normalize the data for display
        rgb = rgb.astype(np.float32)
        for band in range(3):
            min_val = np.percentile(rgb[band], 2)
            max_val = np.percentile(rgb[band], 98)
            rgb[band] = np.clip((rgb[band] - min_val) / (max_val - min_val), 0, 1)
        
        # Transpose to the correct shape for plotting
        rgb = np.transpose(rgb, (1, 2, 0))
        
        # Plot the imagery
        plt.figure(figsize=(10, 10))
        plt.imshow(rgb)
        plt.title(f"Satellite Imagery for Tile {h3_index} (Port: {port})")
        plt.axis('off')
        plt.show()
        
        return rgb

# Visualize imagery for the first result
if results:
    first_result_h3 = results[0].payload["h3_index"]
    visualize_tile_imagery(first_result_h3)

## Compare Multiple Results

Let's compare the satellite imagery for multiple search results.

In [ ]:
def compare_results_imagery(results, num_results=4):
    """Compare the satellite imagery for multiple search results."""
    if not results:
        print("No results to compare")
        return
    
    # Limit the number of results to compare
    results = results[:min(num_results, len(results))]
    
    # Calculate the grid dimensions
    grid_size = int(np.ceil(np.sqrt(len(results))))
    
    # Create a figure with subplots
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    axes = axes.flatten()
    
    # Plot each result
    for i, result in enumerate(results):
        h3_index = result.payload["h3_index"]
        port = result.payload["port"]
        
        # Construct the path to the imagery file
        imagery_path = f"../data/raw/{port}/{h3_index}.tif"
        
        if not os.path.exists(imagery_path):
            axes[i].text(0.5, 0.5, f"No imagery for {h3_index}", ha="center", va="center")
            axes[i].axis('off')
            continue
        
        # Load and display the imagery
        with rasterio.open(imagery_path) as src:
            # Read the first three bands (RGB)
            rgb = src.read([1, 2, 3])
            
            # Normalize the data for display
            rgb = rgb.astype(np.float32)
            for band in range(3):
                min_val = np.percentile(rgb[band], 2)
                max_val = np.percentile(rgb[band], 98)
                rgb[band] = np.clip((rgb[band] - min_val) / (max_val - min_val), 0, 1)
            
            # Transpose to the correct shape for plotting
            rgb = np.transpose(rgb, (1, 2, 0))
            
            # Plot the imagery
            axes[i].imshow(rgb)
            axes[i].set_title(f"Rank {i+1}: {port.capitalize()}\nScore: {result.score:.4f}")
            axes[i].axis('off')
    
    # Hide any unused subplots
    for i in range(len(results), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Compare imagery for multiple results
if results:
    compare_results_imagery(results, num_results=4)

## Custom Search Example: Finding Grain Elevators

Let's create a custom search to find grain elevators across all ports.

In [ ]:
# Define coordinates of known grain elevators
grain_elevator_coordinates = [
    [39.0150, 117.7200],  # Tianjin grain terminal
    [38.9600, 121.6400],  # Dalian grain terminal
    [31.3500, 121.5000]   # Shanghai grain terminal
]

# Search for similar locations
grain_elevator_results = search_by_multiple_examples(grain_elevator_coordinates, limit=15)

# Display results
if grain_elevator_results:
    print("Potential grain elevators found:")
    for i, result in enumerate(grain_elevator_results):
        print(f"Result {i+1}:")
        print(f"  Port: {result.payload['port']}")
        print(f"  H3 Index: {result.payload['h3_index']}")
        print(f"  Similarity Score: {result.score:.4f}")

In [ ]:
# Visualize the grain elevator results on a map
if grain_elevator_results:
    # Create a map centered on China
    m = folium.Map(location=[35, 120], zoom_start=5)
    
    # Add example points (known grain elevators)
    for lat, lng in grain_elevator_coordinates:
        folium.Marker(
            location=[lat, lng],
            popup="Known Grain Elevator",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)
    
    # Add results to the map
    for i, result in enumerate(grain_elevator_results):
        # Skip the first few results as they might be the examples themselves
        if i < len(grain_elevator_coordinates):
            continue
            
        # Get the H3 polygon
        h3_index = result.payload["h3_index"]
        h3_polygon = h3_to_polygon(h3_index)
        
        # Get the center of the H3 cell
        center = h3.cell_to_latlng(h3_index)
        lat, lng = center[0], center[1]
        
        # Add the polygon to the map
        folium.GeoJson(
            h3_polygon,
            style_function=lambda x, score=result.score: {
                "fillColor": "green",
                "color": "green",
                "weight": 2,
                "fillOpacity": min(0.8, max(0.2, score))
            },
            tooltip=f"Potential Grain Elevator, Score: {result.score:.4f}, Port: {result.payload['port']}"
        ).add_to(m)
        
        # Add a marker for the center of the tile
        folium.Marker(
            location=[lat, lng],
            popup=f"Potential Grain Elevator, Score: {result.score:.4f}, Port: {result.payload['port']}",
            icon=folium.Icon(color="green", icon=str(i+1-len(grain_elevator_coordinates)))
        ).add_to(m)
    
    m

In [ ]:
# Compare imagery for the grain elevator results
if grain_elevator_results:
    # Skip the first few results as they might be the examples themselves
    filtered_results = [r for i, r in enumerate(grain_elevator_results) if i >= len(grain_elevator_coordinates)]
    compare_results_imagery(filtered_results, num_results=9)

## Conclusion

In this notebook, we've demonstrated how to use the H3 Satellite Search system to find similar locations based on satellite imagery. We've explored different search methods:

1. Search by coordinates
2. Search by example tile
3. Search by multiple examples
4. Visualize search results

These methods can be used to find specific types of infrastructure (e.g., grain elevators, container terminals) across different ports, or to identify areas with similar visual patterns.